In [22]:
import sympy as sp
from sympy import pi, cos, sin, sqrt, atan2, acos, simplify 
from sympy.matrices import Matrix, eye
from mpmath import degrees, radians

In [10]:
sp.init_printing(use_latex='mathjax')

In [19]:
# from kinematic_analysis import *
# import kinematic_analysis as ka

In [14]:
ka.column((1, 2, ka.Symbol('d')), homogeneous=True)

Matrix([
[1],
[2],
[d],
[1]])

In [17]:
q1, q2, q3, q4, q5, q6 = ka.symbols('q1:7')

In [18]:
# ka.compose_sym_rotations([()])
R_wc_ee = ka.simplify(ka.compose_sym_rotations([('z', q6), ('x', q5), ('z', q4)]))
R_wc_ee

Matrix([
[-sin(q4)*sin(q6)*cos(q5) + cos(q4)*cos(q6), -sin(q4)*cos(q5)*cos(q6) - sin(q6)*cos(q4),  sin(q4)*sin(q5)],
[ sin(q4)*cos(q6) + sin(q6)*cos(q4)*cos(q5), -sin(q4)*sin(q6) + cos(q4)*cos(q5)*cos(q6), -sin(q5)*cos(q4)],
[                           sin(q5)*sin(q6),                            sin(q5)*cos(q6),          cos(q5)]])

In [5]:
def column(spatial_components, homogeneous=False):
    '''
    Convenience function that ensures consistent use of appropriate column-aligned sympy vectors.

    :param spatial_components: ordered iterable of objects compatible with sympy matrices
    :param homogeneous: bool; generate homogeneous coordinate vector by adding a 4th row containing a 1
    '''
    column_matrix = [[item] for item in spatial_components]
    if homogeneous:
        column_matrix.append([1])
    return Matrix(column_matrix)


def sym_rot_x(q):
    '''
    Construct a symbolic matrix corresponding to a rotatation about the x-axis
    '''
    R_x = Matrix([
        [1, 0, 0],
        [0, cos(q), -sin(q)],
        [0, sin(q), cos(q)],
    ])
    return R_x


def sym_rot_y(q):
    R_y = Matrix([
        [cos(q), 0, sin(q)],
        [0, 1, 0],
        [-sin(q), 0, cos(q)],
    ])
    return R_y


def sym_rot_z(q):
    R_z = Matrix([
        [cos(q), -sin(q), 0],
        [sin(q), cos(q), 0],
        [0, 0, 1],
    ])
    return R_z


def compose_sym_rotations(rotations):
    '''
    Compose a sympy rotation matrix corresponding to the sequence of rotations given.
    rotations: ordered iterable (of arbitrary length), which contains iterables of form (axis, radian_angle),
        where axis is an element of {'x', 'y', 'z'}.
    '''
    transform_about = {
        'x': sym_rot_x,
        'y': sym_rot_y,
        'z': sym_rot_z,
    }

    iteratively_composed_matrix = eye(3)

    for rotation in rotations:
        rotation_axis, rotation_angle = rotation[0], rotation[1]
        new_transform = transform_about[rotation_axis](rotation_angle)
        iteratively_composed_matrix = new_transform * iteratively_composed_matrix

    return iteratively_composed_matrix


def sym_tf_matrix(rotation_list, translation):
    '''
    :param rotation_list: rotation sequence as specified by compose_sym_rotations function
    :param translation: iterable of cartesian coords of cumulative translation (x, y, z)
    :return: 4x4 sympy Matrix representing dh transform
    '''
    rot_matrix = compose_sym_rotations(rotation_list)
    return rot_matrix.col_join(Matrix([[0, 0, 0]])).row_join(
        column(translation, homogeneous=True))


def single_dh_transform_step(alpha, a, d, q):
    # alpha, a, d, q = twist_angle, link_length, joint_angle, link_offset
    sub_transform_x = sym_tf_matrix([('x', alpha)], (a, 0, 0))
    sub_transform_z = sym_tf_matrix([('z', q)], (0, 0, d))
    return sp.simplify(sub_transform_x * sub_transform_z)


def rotation_sub_matrix(dh_transform_matrix):
    return dh_transform_matrix[0:3, 0:3]


def translation_sub_matrix(dh_transform_matrix):
    return dh_transform_matrix[0:3, 3]

In [6]:
# Elementary basis vectors x, y, z (non-homogeneous representation)
e_x = column((1, 0, 0))
e_y = column((0, 1, 0))
e_z = column((0, 0, 1))
# Homogeneous elementary basis vectors x, y, z
e_x_h = column((1, 0, 0), homogeneous=True)
e_y_h = column((0, 1, 0), homogeneous=True)
e_z_h = column((0, 0, 1), homogeneous=True)

In [7]:
#######################################
# Create symbols
# symbols for DH parameters
q1, q2, q3, q4, q5, q6, q7 = sp.symbols('q1:8')  # theta angles
d1, d2, d3, d4, d5, d6, d7 = sp.symbols('d1:8')
a0, a1, a2, a3, a4, a5, a6 = sp.symbols('a0:7')
alpha0, alpha1, alpha2, alpha3, alpha4, alpha5, alpha6 = sp.symbols('alpha0:7')

# symbols for end effector target coordinates in base_link reference frame
x_target, y_target, z_target = sp.symbols('x_target y_target z_target')

# Create symbols for wrist center location in base frame
z_wrist, y_wrist, z_wrist = sp.symbols('wrist wrist z_wrist')

# Create Modified DH parameters
# Define the symbol dictionary which contains lookup values for the system
# ATTENTION! Note that joints 4-6 share a common origin, actually located at the wrist center,
# at the intersection of their Z axes.
dh_parameters = {alpha0: 0,              a0: 0,      d1: 0.75,
                 alpha1: -sp.pi / 2,     a1: 0.35,   d2: 0,      q2: q2 - sp.pi / 2,
                 alpha2: 0,              a2: 1.25,   d3: 0,
                 alpha3: -sp.pi / 2,     a3: -0.054, d4: 1.50,
                 alpha4: sp.pi / 2,      a4: 0,      d5: 0,
                 alpha5: -sp.pi / 2,     a5: 0,      d6: 0,
                 alpha6: 0,              a6: 0,      d7: 0.303,  q7: 0
                 }

# Define Modified DH Transformation matrix

In [11]:
# Create individual transformation matrices
T0_1 = single_dh_transform_step(alpha0, a0, d1, q1).subs(dh_parameters)
T1_2 = single_dh_transform_step(alpha1, a1, d2, q2).subs(dh_parameters)
T2_3 = single_dh_transform_step(alpha2, a2, d3, q3).subs(dh_parameters)
T3_4 = single_dh_transform_step(alpha3, a3, d4, q4).subs(dh_parameters)
T4_5 = single_dh_transform_step(alpha4, a4, d5, q5).subs(dh_parameters)
T5_6 = single_dh_transform_step(alpha5, a5, d6, q6).subs(dh_parameters)
T6_7 = single_dh_transform_step(alpha6, a6, d7, q7).subs(dh_parameters)

In [12]:
T0_EE = sp.simplify(T0_1 * T1_2 * T2_3 * T3_4 * T4_5 * T5_6 * T6_7)

In [13]:
T0_EE

⎡((sin(q₁)⋅sin(q₄) + sin(q₂ + q₃)⋅cos(q₁)⋅cos(q₄))⋅cos(q₅) + sin(q₅)⋅cos(q₁)⋅c
⎢                                                                             
⎢((sin(q₁)⋅sin(q₂ + q₃)⋅cos(q₄) - sin(q₄)⋅cos(q₁))⋅cos(q₅) + sin(q₁)⋅sin(q₅)⋅c
⎢                                                                             
⎢                                -(sin(q₅)⋅sin(q₂ + q₃) - cos(q₄)⋅cos(q₅)⋅cos(
⎢                                                                             
⎣                                                                             

os(q₂ + q₃))⋅cos(q₆) - (-sin(q₁)⋅cos(q₄) + sin(q₄)⋅sin(q₂ + q₃)⋅cos(q₁))⋅sin(q
                                                                              
os(q₂ + q₃))⋅cos(q₆) - (sin(q₁)⋅sin(q₄)⋅sin(q₂ + q₃) + cos(q₁)⋅cos(q₄))⋅sin(q₆
                                                                              
q₂ + q₃))⋅cos(q₆) - sin(q₄)⋅sin(q₆)⋅cos(q₂ + q₃)                              
                                                   

In [14]:
R0_1 = T0_1[0:3, 0:3]
R1_2 = T1_2[0:3, 0:3]
R2_3 = T2_3[0:3, 0:3]
R3_4 = T3_4[0:3, 0:3]
R4_5 = T4_5[0:3, 0:3]
R5_6 = T5_6[0:3, 0:3]
R6_7 = T6_7[0:3, 0:3]

In [15]:
R0_EE = sp.simplify(R0_1 * R1_2 * R2_3 * R3_4 * R4_5 * R5_6 * R6_7)

In [16]:
R0_EE

⎡((sin(q₁)⋅sin(q₄) + sin(q₂ + q₃)⋅cos(q₁)⋅cos(q₄))⋅cos(q₅) + sin(q₅)⋅cos(q₁)⋅c
⎢                                                                             
⎢((sin(q₁)⋅sin(q₂ + q₃)⋅cos(q₄) - sin(q₄)⋅cos(q₁))⋅cos(q₅) + sin(q₁)⋅sin(q₅)⋅c
⎢                                                                             
⎣                                -(sin(q₅)⋅sin(q₂ + q₃) - cos(q₄)⋅cos(q₅)⋅cos(

os(q₂ + q₃))⋅cos(q₆) - (-sin(q₁)⋅cos(q₄) + sin(q₄)⋅sin(q₂ + q₃)⋅cos(q₁))⋅sin(q
                                                                              
os(q₂ + q₃))⋅cos(q₆) - (sin(q₁)⋅sin(q₄)⋅sin(q₂ + q₃) + cos(q₁)⋅cos(q₄))⋅sin(q₆
                                                                              
q₂ + q₃))⋅cos(q₆) - sin(q₄)⋅sin(q₆)⋅cos(q₂ + q₃)                              

₆)  -((sin(q₁)⋅sin(q₄) + sin(q₂ + q₃)⋅cos(q₁)⋅cos(q₄))⋅cos(q₅) + sin(q₅)⋅cos(q
                                                                              
)   -((sin(q₁)⋅sin(q₂ + q₃)⋅cos(q₄) - sin(q₄)⋅cos(

In [17]:
R0_EE_from_dhtf = rotation_sub_matrix(T0_EE)

In [18]:
R0_EE == R0_EE_from_dhtf

True

### The following cells show the outcome of using the correction matrix from the walkthrough
using the mpmath radians function rather than sympy's pi object introduces a miniscule amount of error, but why introduce any numerical error at all when the desired result is encoded PERFECTLY by a clean, symbolic solution?

In [54]:
# Conversion Factors - Note that this is using the symbolic definition of pi from SymPy
rtd = 180 / sp.pi  # radians to degrees
dtr = sp.pi / 180  # degrees to radians

r, p, y = sp.symbols('r p y')

R_corr_sym = sym_rot_z(y) * sym_rot_y(p)


# Walkthrough method
R_corr_a = sym_rot_z(y).subs(y, radians(180)) * sym_rot_y(p).subs(p, radians(-90))
# Pure symbolic method
R_corr_b = R_corr_b.subs({y: dtr * 180, p: dtr * -90})

R_corr_sym

⎡cos(p)⋅cos(y)  -sin(y)  sin(p)⋅cos(y)⎤
⎢                                     ⎥
⎢sin(y)⋅cos(p)  cos(y)   sin(p)⋅sin(y)⎥
⎢                                     ⎥
⎣   -sin(p)        0        cos(p)    ⎦

In [55]:
R_corr_a, R_corr_b

⎛⎡-6.12323399573677e-17  -1.22464679914735e-16           1.0         ⎤  ⎡0  0 
⎜⎢                                                                   ⎥  ⎢     
⎜⎢7.49879891330929e-33           -1.0           -1.22464679914735e-16⎥, ⎢0  -1
⎜⎢                                                                   ⎥  ⎢     
⎝⎣         1.0                     0            6.12323399573677e-17 ⎦  ⎣1  0 

  1⎤⎞
   ⎥⎟
  0⎥⎟
   ⎥⎟
  0⎦⎠

In [39]:
R_corr = R_corr_b

In [40]:
R_roll = sym_rot_x(r)
R_pitch = sym_rot_y(p)
R_yaw = sym_rot_z(y)
R_roll, R_pitch, R_yaw

⎛⎡1    0        0   ⎤  ⎡cos(p)   0  sin(p)⎤  ⎡cos(y)  -sin(y)  0⎤⎞
⎜⎢                  ⎥  ⎢                  ⎥  ⎢                  ⎥⎟
⎜⎢0  cos(r)  -sin(r)⎥, ⎢   0     1    0   ⎥, ⎢sin(y)  cos(y)   0⎥⎟
⎜⎢                  ⎥  ⎢                  ⎥  ⎢                  ⎥⎟
⎝⎣0  sin(r)  cos(r) ⎦  ⎣-sin(p)  0  cos(p)⎦  ⎣  0        0     1⎦⎠

In [41]:
R_EE_euler = R_yaw * R_pitch * R_roll * R_corr
R_EE_euler

⎡sin(p)⋅cos(r)⋅cos(y) + sin(r)⋅sin(y)  -sin(p)⋅sin(r)⋅cos(y) + sin(y)⋅cos(r)  
⎢                                                                             
⎢sin(p)⋅sin(y)⋅cos(r) - sin(r)⋅cos(y)  -sin(p)⋅sin(r)⋅sin(y) - cos(r)⋅cos(y)  
⎢                                                                             
⎣           cos(p)⋅cos(r)                         -sin(r)⋅cos(p)              

cos(p)⋅cos(y)⎤
             ⎥
sin(y)⋅cos(p)⎥
             ⎥
   -sin(p)   ⎦

In [43]:
R0_3 = sp.simplify(R0_1 * R1_2 * R2_3)
R0_3

⎡sin(q₂ + q₃)⋅cos(q₁)  cos(q₁)⋅cos(q₂ + q₃)  -sin(q₁)⎤
⎢                                                    ⎥
⎢sin(q₁)⋅sin(q₂ + q₃)  sin(q₁)⋅cos(q₂ + q₃)  cos(q₁) ⎥
⎢                                                    ⎥
⎣    cos(q₂ + q₃)         -sin(q₂ + q₃)         0    ⎦

In [45]:
R0_EE = sp.simplify(R0_3 * R3_4 * R4_5 * R5_6 * R6_7)

In [49]:
R0_3_inverse = sp.simplify(R0_3.inv('LU'))

In [50]:
R0_3_inverse

⎡sin(q₂ + q₃)⋅cos(q₁)  sin(q₁)⋅sin(q₂ + q₃)  cos(q₂ + q₃) ⎤
⎢                                                         ⎥
⎢cos(q₁)⋅cos(q₂ + q₃)  sin(q₁)⋅cos(q₂ + q₃)  -sin(q₂ + q₃)⎥
⎢                                                         ⎥
⎣      -sin(q₁)              cos(q₁)               0      ⎦

And to verify that this result is truly what we were seeking, we can multiply it by R0_3 to check that the result is an identity matrix:

In [53]:
simplify(R0_3 * R0_3_inverse)

⎡1  0  0⎤
⎢       ⎥
⎢0  1  0⎥
⎢       ⎥
⎣0  0  1⎦

This result is totally independent of the pose in question; so long as we can calculate q1, q2, and q3 values, we can determine the numerical-valued form of R0_3_inverse with nothing more than a substitution.

In [57]:
R3_EE = sp.simplify(R0_3_inverse * R0_EE)

In [58]:
R3_EE

⎡                                                                             
⎢-sin(q₄)⋅sin(q₆) + cos(q₄)⋅cos(q₅)⋅cos(q₆)  -sin(q₄)⋅cos(q₆) - sin(q₆)⋅cos(q₄
⎢                                                                             
⎢                                                                             
⎢             sin(q₅)⋅cos(q₆)                             -sin(q₅)⋅sin(q₆)    
⎢                                                                             
⎣-sin(q₄)⋅cos(q₅)⋅cos(q₆) - sin(q₆)⋅cos(q₄)  sin(q₄)⋅sin(q₆)⋅cos(q₅) - cos(q₄)

           sin(q₄ - q₅)   sin(q₄ + q₅)⎤
)⋅cos(q₅)  ──────────── - ────────────⎥
                2              2      ⎥
                                      ⎥
                     cos(q₅)          ⎥
                                      ⎥
⋅cos(q₆)         sin(q₄)⋅sin(q₅)      ⎦

In [143]:
R3_6 = Matrix([
    [-sin(q4)*sin(q6) + cos(q4)*cos(q5)*cos(q6), -sin(q4)*cos(q6) - sin(q6)*cos(q4)*cos(q5), -cos(q4)*sin(q5)],
    [                           sin(q5)*cos(q6),                           -sin(q5)*sin(q6),          cos(q5)],
    [-sin(q4)*cos(q5)*cos(q6) - sin(q6)*cos(q4),  sin(q4)*sin(q6)*cos(q5) - cos(q4)*cos(q6),  sin(q4)*sin(q5)]])
R3_6

⎡-sin(q₄)⋅sin(q₆) + cos(q₄)⋅cos(q₅)⋅cos(q₆)  -sin(q₄)⋅cos(q₆) - sin(q₆)⋅cos(q₄
⎢                                                                             
⎢             sin(q₅)⋅cos(q₆)                             -sin(q₅)⋅sin(q₆)    
⎢                                                                             
⎣-sin(q₄)⋅cos(q₅)⋅cos(q₆) - sin(q₆)⋅cos(q₄)  sin(q₄)⋅sin(q₆)⋅cos(q₅) - cos(q₄)

)⋅cos(q₅)  -sin(q₅)⋅cos(q₄)⎤
                           ⎥
               cos(q₅)     ⎥
                           ⎥
⋅cos(q₆)   sin(q₄)⋅sin(q₅) ⎦

In [76]:
(0.5*(sin(q4 - q5) - sin(q4 + q5))).evalf(subs={q4: 1.1125, q5: 0.3*sp.pi})

-0.357926012846339

In [77]:
(-cos(q4)*sin(q5)).evalf(subs={q4: 1.1125, q5: 0.3*sp.pi})

-0.357926012846339

In [80]:
(-cos(q4)*sin(q5)).evalf(subs={q4: 1.1125, q5: 0.3*sp.pi}) == (0.5*(sin(q4 - q5) - sin(q4 + q5))).evalf(subs={q4: 1.1125, q5: 0.3*sp.pi})

True

In [81]:
r_11 = R3_6[0, 0]
r_12 = R3_6[0, 1]
r_13 = R3_6[0, 2]
r_21 = R3_6[1, 0]
r_22 = R3_6[1, 1]
r_23 = R3_6[1, 2]
r_31 = R3_6[2, 0]
r_32 = R3_6[2, 1]
r_33 = R3_6[2, 2]

The objective here is to come up with formulas for q4, q5, and q6 in terms of particular indexed elements of a 3x3 matrix.
The rotation matrix R3_6_edit, above, is defined in terms of DH parameters, which we are solving for, but the matrix we will draw values from
is the numerical matrix R3_6 = R0_3_inverse.dot(r_EE), where R0_3_inverse is determined by q1-q3, and r_EE is determined by the rpy euler angles from the pose.

In a sense, the above 'r_ij' variables can be considered placeholders for numerical values that will be mapped to joint angles through relationships this matrix encodes.

### Solving q4

In [107]:
r_33, r_13

(sin(q₄)⋅sin(q₅), -sin(q₅)⋅cos(q₄))

In [108]:
r_33 / -r_13

sin(q₄)
───────
cos(q₄)

### Solving q5

In [115]:
r_13, r_33, r_23

(-sin(q₅)⋅cos(q₄), sin(q₄)⋅sin(q₅), cos(q₅))

In [116]:
r_13 ** 2 + r_33 ** 2

   2        2          2        2    
sin (q₄)⋅sin (q₅) + sin (q₅)⋅cos (q₄)

In [131]:
sp.factor(r_13 ** 2 + r_33 ** 2)

⎛   2          2    ⎞    2    
⎝sin (q₄) + cos (q₄)⎠⋅sin (q₅)

In [118]:
sp.simplify(r_13 ** 2 + r_33 ** 2)

   2    
sin (q₅)

In [146]:
sp.sqrt(sp.simplify(r_13 ** 2 + r_33 ** 2)) / r_23

   __________
  ╱    2     
╲╱  sin (q₅) 
─────────────
   cos(q₅)   

In [147]:
sin(q5)/cos(q5)

sin(q₅)
───────
cos(q₅)

In [141]:
q5 == sp.atan2(sp.sqrt(R3_6[0, 2] * R3_6[0, 2] + R3_6[2, 2] * R3_6[2, 2]), R3_6[1, 2])

False

### Solving q6

In [ ]:
theta6 = atan2(-R3_6[1, 1], R3_6[1, 0])

In [136]:
r_22, r_21

(-sin(q₅)⋅sin(q₆), sin(q₅)⋅cos(q₆))

In [140]:
-r_22 / r_21

sin(q₆)
───────
cos(q₆)

Working Result:
```
theta5 = atan2(sqrt(R3_6[0, 2] * R3_6[0, 2] + R3_6[2, 2] * R3_6[2, 2]), R3_6[1, 2]) # r_13**2 + r_33**2, r_23
theta4 = atan2(R3_6[2, 2], -R3_6[0, 2])
theta6 = atan2(-R3_6[1, 1], R3_6[1, 0])
```